In [ ]:
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
import zipfile
import os
import cv2
import torchvision.transforms as T
import torchvision.transforms.functional as FT
from torch.utils.data import DataLoader
from datetime import datetime
import time
import torch.optim
import torch.backends.cudnn as cudnn

In [ ]:
!nvidia-smi

Wed Sep 22 22:25:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
root_dir='/content/drive/MyDrive/osam/dataset'
annotation_dir='/content/drive/MyDrive/osam/LOC_train_solution.csv'
txt_file='/content/drive/MyDrive/osam/sol_train_idx.txt'


val_root_dir='/content/drive/MyDrive/osam/sub'
val_annotation_dir='/content/drive/MyDrive/osam/LOC_val_solution.csv'
val_txt_file='/content/drive/MyDrive/osam/sol_val_idx.txt'

In [ ]:
label_map={'n02687172':0,
'n02916936': 1,
'n02950826': 2,
'n03180011': 3,
'n03594945': 4,
'n03642806': 5,
'n03763968': 6,
'n03773504': 7,
'n03782006': 8,
'n03786901': 9,
'n04008634': 10,
'n04086273': 11,
'n04090263': 12,
'n04389033': 13,
'n04552348': 14,
'n06794110': 15,}

In [ ]:
transform = T.Compose([T.Resize([320,320]), T.ToTensor(),T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

In [ ]:
transform = T.Compose([T.Resize([320,320]),T.ToTensor()])

In [ ]:
epochs_since_improvement = 0  
best_loss = 100.
workers = 2
batch_size=128 #128
print_freq = 1
lr = 1e-3  
momentum = 0.9  
weight_decay = 5e-4  
device='cuda'
torch.manual_seed(1)
torch.cuda.manual_seed(1)
jobs_dir='/content/drive/MyDrive/osam/jobs3'
cudnn.benchmark = True
start_epoch = 0  
checkpoint='/content/drive/MyDrive/osam/jobs3/checkpoint_ssd300.pth.tar006'
num_classes=16 #18+back

In [ ]:
def save_checkpoint(epoch, epochs_since_improvement, model, optimizer, loss, best_loss, is_best, jobs_dir):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param epochs_since_improvement: number of epochs since last improvement
    :param model: model
    :param optimizer: optimizer
    :param loss: validation loss in this epoch
    :param best_loss: best validation loss achieved so far (not necessarily in this checkpoint)
    :param is_best: is this checkpoint the best so far?
    """
    state = {'epoch': epoch,
             'epochs_since_improvement': epochs_since_improvement,
             'loss': loss,
             'best_loss': best_loss,
             'model': model,
             'optimizer': optimizer}
    filename = os.path.join(jobs_dir,'checkpoint_ssd300.pth.tar{:03d}'.format(epoch))
    filename_pt = 'checkpoint_ssd300.pth.tar'
    torch.save(state, filename)
    # If this checkpoint is the best so far, store a copy so it doesn't get overwritten by a worse checkpoint
    if is_best:
        torch.save(state, jobs_dir+'BEST_' + filename_pt)

In [ ]:
def collate_fn(batch):

    """
    Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).
    This describes how to combine these tensors of different sizes. We use lists.
    Note: this need not be defined in this Class, can be standalone.
    :param batch: an iterable of N sets from __getitem__()
    :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
    """

    images = list()
    boxes = list()
    labels = list()


    for b in batch:
        images.append(b[0])
        boxes.append(b[1])
        labels.append(b[2])
        

    images = torch.stack(images, dim=0)
    
    return images, boxes, labels

In [ ]:
class ImageNetDataset(Dataset):
    

    def __init__(self,txt_file, root_dir,annotation_dir, transform=None,mode=None):
        """
        Args:
            
            root_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform

        """
        f = open(txt_file,'r')
        lines=f.readlines()
        lines=list(map(lambda s: s.strip(), lines))

        self.idx_txt=lines
        self.annotation_dir=annotation_dir
        self.root_dir = root_dir
        self.transform = transform

        self.mode=mode

    def __len__(self):
        
        return len(self.idx_txt)

    def __getitem__(self, idx):
      
      if self.mode=='TRAIN':
        img_name=self.idx_txt[idx].split(" ")[0]
        folder_name=img_name.split("_")[0]
        img_path=os.path.join(self.root_dir,folder_name,img_name+'.JPEG')
        img = cv2.imread(img_path)  #(height, width, channel) 
        
        

        solution=pd.read_csv(self.annotation_dir)
        
        cond=solution['ImageId'] == img_name
        ans_ls=solution[cond]['PredictionString'].item().split(' ')
        ans_ls=ans_ls[:len(ans_ls)-1]
        # ans_ls=ans_ls[:5]#For example, n01978287 240 170 260 240 means it's label n01978287, with a bounding box of coordinates (x_min, y_min, x_max, y_max)

        boxes=[]
        labels=[]
        # d={}
        # labels.append(label_map[ans_ls[0]])
        # boxes.append(ans_ls[1:])
        # import pdb;pdb.set_trace()
        for i in range(int(len(ans_ls)/5)):
          labels.append(label_map[ans_ls[5*i]])
          boxes.append(ans_ls[5*i+1:5*i+5])
          
          
        labels=torch.tensor(labels,dtype=torch.int64)
        boxes=torch.FloatTensor(np.array(boxes,dtype=np.float64))
        
        height=img.shape[0]
        width=img.shape[1]

        boxes[0][0]=boxes[0][0]*(320/width)
        boxes[0][2]=boxes[0][2]*(320/width)
        
        boxes[0][1]=boxes[0][1]*(320/height)
        boxes[0][3]=boxes[0][3]*(320/height)   
        
      elif self.mode=='VAL':
        
        img_name=self.idx_txt[idx].split(" ")[0]
        
        img_path=os.path.join(self.root_dir,img_name+'.JPEG')
        img = cv2.imread(img_path)  #(height, width, channel) 
        
        solution=pd.read_csv(self.annotation_dir)
        
        cond=solution['ImageId'] == img_name
        ans_ls=solution[cond]['PredictionString'].item().split(' ')
        ans_ls=ans_ls[:len(ans_ls)-1]
        # ans_ls=ans_ls[:5]#For example, n01978287 240 170 260 240 means it's label n01978287, with a bounding box of coordinates (x_min, y_min, x_max, y_max)

        boxes=[]
        labels=[]
        # d={}
        # labels.append(label_map[ans_ls[0]])
        # boxes.append(ans_ls[1:])
        # import pdb;pdb.set_trace()
        for i in range(int(len(ans_ls)/5)):
          labels.append(label_map[ans_ls[5*i]])
          boxes.append(ans_ls[5*i+1:5*i+5])
          
        labels=torch.tensor(labels,dtype=torch.int64)
        boxes=torch.FloatTensor(np.array(boxes,dtype=np.float64))

        height=img.shape[0]
        width=img.shape[1]

        boxes[0][0]=boxes[0][0]*(320/width)
        boxes[0][2]=boxes[0][2]*(320/width)
        
        boxes[0][1]=boxes[0][1]*(320/height)
        boxes[0][3]=boxes[0][3]*(320/height)       
        # d['boxes']=boxes
        # d['labels']=labels
      else:
        import pdb;pdb.set_trace()
#         The input to the model is expected to be a list of tensors, each of shape [C, H, W], one for each image, and should be in 0-1 range. Different images can have different sizes.

# The behavior of the model changes depending if it is in training or evaluation mode.

# During training, the model expects both the input tensors, as well as a targets (list of dictionary), containing:

# boxes (FloatTensor[N, 4]): the ground-truth boxes in [x1, y1, x2, y2] format, with 0 <= x1 < x2 <= W and 0 <= y1 < y2 <= H.

# labels (Int64Tensor[N]): the class label for each ground-truth box
        
      if self.transform:
          img=FT.to_pil_image(img)
          img = self.transform(img)
        
        
      return img,boxes,labels

In [ ]:
def main():
    """
    Training and validation.
    """
    global epochs_since_improvement, start_epoch, label_map, best_loss, epoch, checkpoint, decay_lr_at

    # Initialize model or load checkpoint
    if checkpoint is None:
        model = torchvision.models.detection.ssd300_vgg16(num_classes=16,pretrained_backbone=False,
        image_mean= [0.485, 0.456, 0.406],
        image_std= [0.229, 0.224, 0.225],)
        __all__ = ['SSD', 'ssd300_vgg16']

        model_urls = {
            'ssd300_vgg16_coco': 'https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth',
        }

        backbone_urls = {
            # We port the features of a VGG16 backbone trained by amdegroot because unlike the one on TorchVision, it uses the
            # same input standardization method as the paper. Ref: https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth
            'vgg16_features': 'https://download.pytorch.org/models/vgg16_features-amdegroot.pth'
        }
        weights_name = 'ssd300_vgg16_coco'
        if model_urls.get(weights_name, None) is None:
            raise ValueError("No checkpoint is available for model {}".format(weights_name))
        state_dict = torch.hub.load_state_dict_from_url(model_urls[weights_name], progress=True)

        except_params=[]
        for i in range(6):
          data1='head.classification_head.module_list.'+str(i)+'.weight'
          data2='head.classification_head.module_list.'+str(i)+'.bias'
          except_params.append(data1)
          except_params.append(data2)

          
        for param_name, param in model.named_parameters():
          

          if param_name in except_params:
            # param=torch.zeros(state_dict[param_name].shape)
            state_dict[param_name]=param
            # continue

          else:
            continue
            # param=torch.zeros(param.shape)
            # state_dict[param_name]=param
        model.load_state_dict(state_dict)   
        # model.load_state_dict(state_dict)
        # model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(num_classes=num_classes,pretrained_backbone=True)
        # model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)
        # Initialize the optimizer, with twice the default learning rate for biases, as in the original Caffe repo
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
          # biases = list()
          # not_biases = list()
          if param.requires_grad:
              if param_name.endswith('.bias'):
                  biases.append(param)
              else:
                  not_biases.append(param)
        optimizer = torch.optim.SGD(params=[{'params': biases, 'lr': 2 * lr}, {'params': not_biases}],
                                    lr=lr, momentum=momentum, weight_decay=weight_decay)
        
        
    else:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        epochs_since_improvement = checkpoint['epochs_since_improvement']
        best_loss = checkpoint['best_loss']
        print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
        
        
    # Move to default device
    model = model.to(device)
    

    # Custom dataloaders
    train_dataset = ImageNetDataset(txt_file,root_dir,annotation_dir,transform=transform,mode='TRAIN')
        
    train_loader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=workers,collate_fn=collate_fn) # note that we're passing the collate function here
    
    val_dataset = ImageNetDataset(val_txt_file,val_root_dir,val_annotation_dir,transform=transform,mode='VAL')

    val_loader =  DataLoader(val_dataset, batch_size=batch_size, shuffle=True,num_workers=workers,collate_fn=collate_fn) # note that we're passing the collate function here


    for epoch in range(start_epoch,300):
        
        
        
        
        # One epoch's validation
        train(train_loader=train_loader,model=model,optimizer=optimizer,epoch=epoch)
        val_loss = validate(val_loader=val_loader,
                            model=model)
        
        # Did validation loss improve?
        is_best = val_loss < best_loss
        best_loss = min(val_loss, best_loss)
        
        if not is_best:
            
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))

        else:
            
            epochs_since_improvement = 0
        if epoch % 1 ==0:    
          # Save checkpoint
          save_checkpoint(epoch, epochs_since_improvement, model, optimizer, val_loss, best_loss, is_best,jobs_dir)

In [ ]:
def train(train_loader, model,optimizer, epoch):

  model.train()  # training mode enables dropout

  batch_time = AverageMeter()  # forward prop. + back prop. time
  data_time = AverageMeter()  # data loading time
  losses = AverageMeter()  # loss

  start = time.time()
  
  for i, (images, boxes,labels) in enumerate(train_loader):
    
    images = images.to(device)
    
    targets=[]
    
    for j in range(images.shape[0]):
      d={}
      d['boxes'] = boxes[j].to(device)
      d['labels'] = labels[j].to(device)
      targets.append(d)
    
    out=model(images,targets)
    
    
    loss=out['bbox_regression']+out['classification']
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    losses.update(loss.item(), images.size(0))
    batch_time.update(time.time() - start)

    start = time.time()

    
    # targets=[targets]
    # targets[0]['labels']=targets[0]['labels'].squeeze(0)
    if i % print_freq == 0:
              print(out['bbox_regression'],out['classification'])
              print('Epoch: [{0}][{1}/{2}]\t'
                    'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                                    batch_time=batch_time,
                                                                    data_time=data_time, loss=losses))
  del out,images, boxes, labels  # free some memory since their histories may be stored


In [ ]:
def validate(val_loader, model):
    
    # model.eval()  # eval mode disables dropout

    batch_time = AverageMeter()
    losses = AverageMeter()

    start = time.time()

    # Prohibit gradient computation explicity because I had some problems with memory
    with torch.no_grad():
        # Batches
        for i, (images, boxes, labels) in enumerate(val_loader):

            # Move to default device
            images = images.to(device)
    
            targets=[]
            
            for j in range(images.shape[0]):
              d={}
              d['boxes'] = boxes[j].to(device)
              d['labels'] = labels[j].to(device)
              targets.append(d)
            
            out=model(images,targets)
            
            loss=out['bbox_regression']+out['classification']

            losses.update(loss.item(), images.size(0))
            batch_time.update(time.time() - start)

            start = time.time()

            # Print status
            if i % print_freq == 0:
                print(out['bbox_regression'],out['classification'])
                print('[{0}/{1}]\t'
                      'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(i, len(val_loader),
                                                                      batch_time=batch_time,
                                                                      loss=losses))

    print('\n * LOSS - {loss.avg:.3f}\n'.format(loss=losses))
    
    return losses.avg

In [ ]:
if __name__ == '__main__':
    main()

**Previous Log**

In [ ]:


if __name__ == '__main__':
    main()

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


  0%|          | 0.00/21.1M [00:00<?, ?B/s]

Epoch: [0][0/59]	Batch Time 203.894 (203.894)	Data Time 0.000 (0.000)	Loss 6.2508 (6.2508)	
Epoch: [0][1/59]	Batch Time 1.432 (102.663)	Data Time 0.000 (0.000)	Loss 6.1933 (6.2220)	
Epoch: [0][2/59]	Batch Time 164.841 (123.389)	Data Time 0.000 (0.000)	Loss 6.2763 (6.2401)	
Epoch: [0][3/59]	Batch Time 1.273 (92.860)	Data Time 0.000 (0.000)	Loss 6.1811 (6.2254)	
Epoch: [0][4/59]	Batch Time 163.101 (106.908)	Data Time 0.000 (0.000)	Loss 6.4040 (6.2611)	
Epoch: [0][5/59]	Batch Time 1.214 (89.292)	Data Time 0.000 (0.000)	Loss 6.5127 (6.3030)	
Epoch: [0][6/59]	Batch Time 167.122 (100.411)	Data Time 0.000 (0.000)	Loss 6.3608 (6.3113)	
Epoch: [0][7/59]	Batch Time 1.275 (88.019)	Data Time 0.000 (0.000)	Loss 6.1284 (6.2884)	
Epoch: [0][8/59]	Batch Time 159.400 (95.950)	Data Time 0.000 (0.000)	Loss 6.0959 (6.2670)	
Epoch: [0][9/59]	Batch Time 7.225 (87.078)	Data Time 0.000 (0.000)	Loss 6.2285 (6.2632)	
Epoch: [0][10/59]	Batch Time 163.629 (94.037)	Data Time 0.000 (0.000)	Loss 6.5638 (6.2905)	
Epo